In [1]:
import pandas as pd
import json
import os
import numpy as np

pd.options.display.max_columns = 999

### TRACKING DATA AND EVENTS

In [2]:
# load tracking data
current_directory = os.getcwd()
path_tracking = os.path.join(os.path.join(current_directory, 'data'), "tracking_set_0")
#print(path_tracking)
game_id = 1

df_tracking = pd.read_parquet(f'{path_tracking}/{game_id}_tracking.parquet')  

In [3]:
# load events names
path_event_csv = os.path.join(current_directory,'data')
df_event_names = pd.read_csv(os.path.join(path_event_csv,'event_names.csv'))

dict_event_names = df_event_names.set_index('event_type_id').to_dict()['event_description']

In [4]:
# load event data
def load_event_data(file_name, base_path):
    # read in event file
    with open(f'{base_path}/{file_name}') as f:
        data=json.loads(f.read())

    f.close()
    
    # transform data into pandas dataframe
    df_events = pd.json_normalize(data['liveData']['event'])
    
    # preprocess event data and keep relevant information only

    # add timeelapsed to each event
    df_events['timestamp'] = pd.to_datetime(df_events.timeStamp).apply(lambda x: x.timestamp())

    df_events = df_events.query('periodId in [1,2]')

    def add_timeelapsed_to_events(df):
        start_time = df.query('typeId==32')['timestamp'].iloc[0]
        df['timestamp_new'] = np.int64((df['timestamp'] - start_time)*1000)

        df['timeelapsed'] = df['timestamp_new'].apply(lambda x: (40 * round(x/40))/1000)

        return df

    df_events = df_events.groupby('periodId').apply(add_timeelapsed_to_events)

    df_events = df_events.drop(columns=['timeStamp','timestamp','timestamp_new'])
    
    # rename some columns
    df_events = df_events.rename(columns=
        {
            'periodId':'current_phase',
            'typeId':'event_type_id',
            'timeMin':'period_minute',
            'timeSec':'period_second'
        }
    )
    
    return df_events

In [5]:
path_events = os.path.join(os.path.join(current_directory,'data'),"first_10_events")
#print(path_events)

event_file = f'{game_id}.json'

df_events = load_event_data(
    base_path=path_events,
    file_name=event_file
)

# add event descriptions
df_events['event_description'] = df_events['event_type_id'].map(dict_event_names)

# make a copy of it for later usage
events_all = df_events.copy()

C:\Users\alber\AppData\Local\Temp\ipykernel_23040\3763987763.py:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_events = df_events.groupby('periodId').apply(add_timeelapsed_to_events)


In [6]:
display(df_events.columns)

Index(['id', 'eventId', 'event_type_id', 'current_phase', 'period_minute',
       'period_second', 'contestantId', 'outcome', 'x', 'y', 'lastModified',
       'qualifier', 'playerId', 'lineBreakingPass.linesBroken.value',
       'passOption.player', 'passTarget.player', 'xThreat.applied',
       'lineBreakingPass.lastLineBroken.value',
       'pressure.pressureReceived.value', 'pressure.player', 'xThreat.removed',
       'keyPass', 'assist', 'timeelapsed', 'event_description'],
      dtype='object')

### QUALIFIERS

In [7]:
# Read in qualifier list
path_data = os.path.join(current_directory,'data')
qualifier_names = pd.read_csv(os.path.join(path_data,"qualifier_names.csv"))
#display(qualifier_names)

# Explode coverts each element in each list to a separate row
cols = ['id', 'qualifier']
qualifiers = events_all[cols].explode('qualifier')
#display(qualifiers.head())

qualifiers = qualifiers[qualifiers.qualifier.notna()].reset_index(drop=True)
#print(qualifiers.shape)
#display(qualifiers.head())

# save corresponding event ids for each qualifier
event_ids = qualifiers.id.tolist()

qualifiers = pd.json_normalize(qualifiers[qualifiers.qualifier.notna()]['qualifier'])
#print(qualifiers.shape)
#display(qualifiers.head())

qualifiers['event_id'] = event_ids
#display(qualifiers.head())

qualifiers = qualifiers.merge(qualifier_names, how='left', on='qualifierId')

In [8]:
display(qualifiers.head())

,id,qualifierId,value,event_id,qualifier
0,3586084711,127,Right to Left,2423549045,Direction of Play
1,3586084701,127,Left to Right,2423549041,Direction of Play
2,3586084825,56,Back,2423549063,Zone
3,3586084833,213,2.7,2423549063,Angle
4,3586084827,140,28.5,2423549063,Pass End X


## MÈTRIQUES

In [60]:
display(df_events.head())

,id,eventId,event_type_id,current_phase,period_minute,period_second,contestantId,outcome,x,y,lastModified,qualifier,playerId,lineBreakingPass.linesBroken.value,passOption.player,passTarget.player,xThreat.applied,lineBreakingPass.lastLineBroken.value,pressure.pressureReceived.value,pressure.player,xThreat.removed,keyPass,assist,timeelapsed,event_description
2,2423549045,2,32,1,0,0,3c3jcs7vc1t6vz5lev162jyv7,1,0.0,0.0,2022-05-22T03:17:52Z,"[{'id': 3586084711, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,Period start
3,2423549041,2,32,1,0,0,bx0cdmzr2gwr70ez72dorx82p,1,0.0,0.0,2022-05-21T18:59:34Z,"[{'id': 3586084701, 'qualifierId': 127, 'value...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,Period start
4,2423549063,3,1,1,0,0,bx0cdmzr2gwr70ez72dorx82p,1,49.9,50.0,2022-05-22T03:34:41Z,"[{'id': 3586084825, 'qualifierId': 56, 'value'...",6u2ob6fv950r1qve8uejkq2uh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04,Pass
5,2423549097,4,1,1,0,2,bx0cdmzr2gwr70ez72dorx82p,1,31.5,57.2,2022-05-22T06:37:07Z,"[{'id': 3586085043, 'qualifierId': 213, 'value...",azuc3tma44xyrbgf5y279o1xx,0,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...","[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",0.0029771626,NaN,NaN,NaN,NaN,NaN,NaN,2.84,Pass
6,2423549113,5,1,1,0,7,bx0cdmzr2gwr70ez72dorx82p,1,49.2,95.4,2022-05-22T06:37:06Z,"[{'id': 3586085129, 'qualifierId': 212, 'value...",7sep6mx2s67mh5fr3raxu7aei,1,"[{'playerId': '5qgc6zjc38a5xjl35gs7h3vu1', 'sh...","[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0309752524,secondToLast,high,"[{'playerId': 'e6ok0deqkoe80184iu509gzu2', 'sh...",NaN,NaN,NaN,7.88,Pass


In [9]:
def event_per_player(df, event, name, sort_by='playerId', ascending=True):
    df = df.dropna(subset=['playerId'])
    try:
        df = df[df['event_description'] == event]
        result_df = df.groupby('playerId').size().reset_index(name=name)
        result_df = result_df.sort_values(by=sort_by, ascending=ascending)
        result_df = result_df.reset_index(drop=True)
        return result_df
    except:
        print(f"Error: 'event_description' column not found in the DataFrame.")
        return False

### Passes completats, intentats, totals, % success, keyPasses per jugador

In [10]:
df_passes = df_events[df_events['event_description']=='Pass']

In [11]:
passes_totals = event_per_player(df_passes,'Pass', 'Total_passes')

In [12]:
df_passes_completed = df_passes[df_passes['outcome']==1]
passes_completats = event_per_player(df_passes_completed,'Pass', 'Passes_completed')
#print(passes_completats)

In [13]:
df_passes_failed = df_passes[df_passes['outcome']==0]
passes_failed = event_per_player(df_passes_failed,'Pass', 'Passes_failed')
#print(passes_failed)

In [14]:
merged_df_passes = pd.merge(passes_totals, passes_completats, on='playerId', how='left')
merged_df_passes = pd.merge(merged_df_passes, passes_failed, on='playerId', how='left')
# Fill NaN values with 0 (for players who didn't have passes completed or failed)
merged_df_passes = merged_df_passes.fillna(0)
merged_df_passes[['Passes_completed', 'Passes_failed']] = merged_df_passes[['Passes_completed', 'Passes_failed']].astype(int)
merged_df_passes['%_pass_success'] = round((merged_df_passes['Passes_completed'] / merged_df_passes['Total_passes']) * 100,2)

In [15]:
display(merged_df_passes)

,playerId,Total_passes,Passes_completed,Passes_failed,%_pass_success
0,2lvit204llltk13iglsa2tjah,2,1,1,50.00
1,3sc349yey596xp2j6xlyt0frp,54,44,10,81.48
2,3vx94h32ahujciraspdayj9t6,19,17,2,89.47
3,4u281v53ges3kimtgac0tidm2,52,42,10,80.77
4,5ak9fwtqlr2pll0nsv5br7p7u,20,12,8,60.00
5,5qgc6zjc38a5xjl35gs7h3vu1,36,26,10,72.22
6,6ekdnbnk56xlxforb5owt3dn9,47,39,8,82.98
7,6j0ogojh2b7poyceg7i3k09yi,64,58,6,90.62
8,6u2ob6fv950r1qve8uejkq2uh,59,50,9,84.75
9,72d5uxwcmvhd6mzthxuvev1sl,38,36,2,94.74


In [16]:
df_keyPasses = df_passes[df_passes['keyPass']==True]

In [17]:
keyPasses = event_per_player(df_keyPasses, 'Pass', 'keyPasses')
display(keyPasses)

,playerId,keyPasses
0,5ak9fwtqlr2pll0nsv5br7p7u,1
1,6j0ogojh2b7poyceg7i3k09yi,1
2,6u2ob6fv950r1qve8uejkq2uh,1
3,7sep6mx2s67mh5fr3raxu7aei,3
4,8gkexxgf3pypshhqwg6ibp7o4,1
5,96wcx761pzv5ub4sfwsynp51x,1
6,976riwm0dz0e74d4l28y3ttcl,1
7,bvbebtykj45j3luvemk8yc4ph,1
8,vja0xo3xiuax8eh0b6q3y09,1


In [18]:
# Merge key passes with the previously merged DataFrame
new_df_passes = pd.merge(merged_df_passes, keyPasses, on='playerId', how='left')
new_df_passes['keyPasses'] = new_df_passes['keyPasses'].fillna(0).astype(int)

In [19]:
# Display the resulting DataFrame
display(new_df_passes)

,playerId,Total_passes,Passes_completed,Passes_failed,%_pass_success,keyPasses
0,2lvit204llltk13iglsa2tjah,2,1,1,50.00,0
1,3sc349yey596xp2j6xlyt0frp,54,44,10,81.48,0
2,3vx94h32ahujciraspdayj9t6,19,17,2,89.47,0
3,4u281v53ges3kimtgac0tidm2,52,42,10,80.77,0
4,5ak9fwtqlr2pll0nsv5br7p7u,20,12,8,60.00,1
5,5qgc6zjc38a5xjl35gs7h3vu1,36,26,10,72.22,0
6,6ekdnbnk56xlxforb5owt3dn9,47,39,8,82.98,0
7,6j0ogojh2b7poyceg7i3k09yi,64,58,6,90.62,1
8,6u2ob6fv950r1qve8uejkq2uh,59,50,9,84.75,1
9,72d5uxwcmvhd6mzthxuvev1sl,38,36,2,94.74,0


In [20]:
new_df_passes.columns

Index(['playerId', 'Total_passes', 'Passes_completed', 'Passes_failed',
       '%_pass_success', 'keyPasses'],
      dtype='object')

### Passes and Pressure Received

In [21]:
df_passes_pressure = df_passes.copy()
df_passes_pressure['pressure_received'] = df_passes_pressure['pressure.pressureReceived.value'].fillna('NaN')
df_passes_pressure = df_passes_pressure.drop('pressure.pressureReceived.value', axis=1)

In [22]:
display(df_passes_pressure.head(3))

,id,eventId,event_type_id,current_phase,period_minute,period_second,contestantId,outcome,x,y,lastModified,qualifier,playerId,lineBreakingPass.linesBroken.value,passOption.player,passTarget.player,xThreat.applied,lineBreakingPass.lastLineBroken.value,pressure.player,xThreat.removed,keyPass,assist,timeelapsed,event_description,pressure_received
4,2423549063,3,1,1,0,0,bx0cdmzr2gwr70ez72dorx82p,1,49.9,50.0,2022-05-22T03:34:41Z,"[{'id': 3586084825, 'qualifierId': 56, 'value'...",6u2ob6fv950r1qve8uejkq2uh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04,Pass,NaN
5,2423549097,4,1,1,0,2,bx0cdmzr2gwr70ez72dorx82p,1,31.5,57.2,2022-05-22T06:37:07Z,"[{'id': 3586085043, 'qualifierId': 213, 'value...",azuc3tma44xyrbgf5y279o1xx,0,"[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...","[{'playerId': '7sep6mx2s67mh5fr3raxu7aei', 'sh...",0.0029771626,NaN,NaN,NaN,NaN,NaN,2.84,Pass,NaN
6,2423549113,5,1,1,0,7,bx0cdmzr2gwr70ez72dorx82p,1,49.2,95.4,2022-05-22T06:37:06Z,"[{'id': 3586085129, 'qualifierId': 212, 'value...",7sep6mx2s67mh5fr3raxu7aei,1,"[{'playerId': '5qgc6zjc38a5xjl35gs7h3vu1', 'sh...","[{'playerId': 'e3kdoxu1kwn2w3wwi1rqhvr9x', 'sh...",0.0309752524,secondToLast,"[{'playerId': 'e6ok0deqkoe80184iu509gzu2', 'sh...",NaN,NaN,NaN,7.88,Pass,high


#### Passes Pressure NaN

In [23]:
df_pp_NaN = df_passes_pressure[df_passes_pressure['pressure_received']=='NaN']

In [24]:
passes_totals_NaN = event_per_player(df_pp_NaN,'Pass', 'Total_passes_NaN')

df_pp_NaN_completats = df_pp_NaN[df_pp_NaN['outcome']==1]
passes_completats_NaN = event_per_player(df_pp_NaN_completats,'Pass', 'Passes_completed_NaN')

df_pp_NaN_fallats = df_pp_NaN[df_pp_NaN['outcome']==0]
passes_fallats_NaN = event_per_player(df_pp_NaN_fallats,'Pass', 'Passes_fallats_NaN')

In [25]:
merged_df_passes_NaN = pd.merge(passes_totals_NaN, passes_completats_NaN, on='playerId', how='left')
merged_df_passes_NaN = pd.merge(merged_df_passes_NaN, passes_fallats_NaN, on='playerId', how='left')
# Fill NaN values with 0 (for players who didn't have passes completed or failed)
merged_df_passes_NaN = merged_df_passes_NaN.fillna(0)
merged_df_passes_NaN[['Passes_completed_NaN', 'Passes_fallats_NaN']] = merged_df_passes_NaN[['Passes_completed_NaN', 'Passes_fallats_NaN']].astype(int)
merged_df_passes_NaN['%_pass_success_NaN'] = round((merged_df_passes_NaN['Passes_completed_NaN'] / merged_df_passes_NaN['Total_passes_NaN']) * 100,2)

In [26]:
display(merged_df_passes_NaN)

,playerId,Total_passes_NaN,Passes_completed_NaN,Passes_fallats_NaN,%_pass_success_NaN
0,2lvit204llltk13iglsa2tjah,1,1,0,100.00
1,3sc349yey596xp2j6xlyt0frp,21,17,4,80.95
2,3vx94h32ahujciraspdayj9t6,4,4,0,100.00
3,4u281v53ges3kimtgac0tidm2,20,18,2,90.00
4,5ak9fwtqlr2pll0nsv5br7p7u,5,4,1,80.00
5,5qgc6zjc38a5xjl35gs7h3vu1,7,5,2,71.43
6,6ekdnbnk56xlxforb5owt3dn9,44,37,7,84.09
7,6j0ogojh2b7poyceg7i3k09yi,15,14,1,93.33
8,6u2ob6fv950r1qve8uejkq2uh,7,7,0,100.00
9,72d5uxwcmvhd6mzthxuvev1sl,23,21,2,91.30


#### Passes Pressure Low

In [27]:
df_pp_Low = df_passes_pressure[df_passes_pressure['pressure_received']=='low']

In [28]:
passes_totals_Low = event_per_player(df_pp_Low,'Pass', 'Total_passes_Low')

df_pp_Low_completats = df_pp_Low[df_pp_Low['outcome']==1]
passes_completats_Low = event_per_player(df_pp_Low_completats,'Pass', 'Passes_completed_Low')

df_pp_Low_fallats = df_pp_Low[df_pp_Low['outcome']==0]
passes_fallats_Low = event_per_player(df_pp_Low_fallats,'Pass', 'Passes_fallats_Low')

In [29]:
merged_df_passes_Low = pd.merge(passes_totals_Low, passes_completats_Low, on='playerId', how='left')
merged_df_passes_Low = pd.merge(merged_df_passes_Low, passes_fallats_Low, on='playerId', how='left')
# Fill NaN values with 0 (for players who didn't have passes completed or failed)
merged_df_passes_Low = merged_df_passes_Low.fillna(0)
merged_df_passes_Low[['Passes_completed_Low', 'Passes_fallats_Low']] = merged_df_passes_Low[['Passes_completed_Low', 'Passes_fallats_Low']].astype(int)
merged_df_passes_Low['%_pass_success_Low'] = round((merged_df_passes_Low['Passes_completed_Low'] / merged_df_passes_Low['Total_passes_Low']) * 100,2)

In [30]:
display(merged_df_passes_Low)

,playerId,Total_passes_Low,Passes_completed_Low,Passes_fallats_Low,%_pass_success_Low
0,3vx94h32ahujciraspdayj9t6,1,1,0,100.00
1,4u281v53ges3kimtgac0tidm2,4,4,0,100.00
2,5ak9fwtqlr2pll0nsv5br7p7u,2,1,1,50.00
3,5qgc6zjc38a5xjl35gs7h3vu1,3,2,1,66.67
4,6j0ogojh2b7poyceg7i3k09yi,8,8,0,100.00
5,6u2ob6fv950r1qve8uejkq2uh,9,7,2,77.78
6,72d5uxwcmvhd6mzthxuvev1sl,1,1,0,100.00
7,7cp51c8zn7y08iyk0hc9ix5nt,10,10,0,100.00
8,7sep6mx2s67mh5fr3raxu7aei,6,4,2,66.67
9,8f3bhiy6r5eei1n25exhbwr8p,6,3,3,50.00


#### Passes Pressure Medium

In [31]:
df_pp_Medium = df_passes_pressure[df_passes_pressure['pressure_received']=='medium']

In [32]:
passes_totals_Medium = event_per_player(df_pp_Medium,'Pass', 'Total_passes_Medium')

df_pp_Medium_completats = df_pp_Medium[df_pp_Medium['outcome']==1]
passes_completats_Medium = event_per_player(df_pp_Medium_completats,'Pass', 'Passes_completed_Medium')

df_pp_Medium_fallats = df_pp_Medium[df_pp_Medium['outcome']==0]
passes_fallats_Medium = event_per_player(df_pp_Medium_fallats,'Pass', 'Passes_fallats_Medium')

In [33]:
merged_df_passes_Medium = pd.merge(passes_totals_Medium, passes_completats_Medium, on='playerId', how='left')
merged_df_passes_Medium = pd.merge(merged_df_passes_Medium, passes_fallats_Medium, on='playerId', how='left')
# Fill NaN values with 0 (for players who didn't have passes completed or failed)
merged_df_passes_Medium = merged_df_passes_Medium.fillna(0)
merged_df_passes_Medium[['Passes_completed_Medium', 'Passes_fallats_Medium']] = merged_df_passes_Medium[['Passes_completed_Medium', 'Passes_fallats_Medium']].astype(int)
merged_df_passes_Medium['%_pass_success_Medium'] = round((merged_df_passes_Medium['Passes_completed_Medium'] / merged_df_passes_Medium['Total_passes_Medium']) * 100,2)

In [34]:
display(merged_df_passes_Medium)

,playerId,Total_passes_Medium,Passes_completed_Medium,Passes_fallats_Medium,%_pass_success_Medium
0,3sc349yey596xp2j6xlyt0frp,13,11,2,84.62
1,3vx94h32ahujciraspdayj9t6,3,2,1,66.67
2,4u281v53ges3kimtgac0tidm2,11,10,1,90.91
3,5ak9fwtqlr2pll0nsv5br7p7u,3,2,1,66.67
4,5qgc6zjc38a5xjl35gs7h3vu1,7,6,1,85.71
5,6ekdnbnk56xlxforb5owt3dn9,1,1,0,100.00
6,6j0ogojh2b7poyceg7i3k09yi,13,12,1,92.31
7,6u2ob6fv950r1qve8uejkq2uh,18,16,2,88.89
8,72d5uxwcmvhd6mzthxuvev1sl,4,4,0,100.00
9,7cp51c8zn7y08iyk0hc9ix5nt,12,10,2,83.33


#### Passes Pressure High

In [35]:
df_pp_High = df_passes_pressure[df_passes_pressure['pressure_received']=='high']

In [36]:
passes_totals_High = event_per_player(df_pp_High,'Pass', 'Total_passes_High')

df_pp_High_completats = df_pp_High[df_pp_High['outcome']==1]
passes_completats_High = event_per_player(df_pp_High_completats,'Pass', 'Passes_completed_High')

df_pp_High_fallats = df_pp_High[df_pp_High['outcome']==0]
passes_fallats_High = event_per_player(df_pp_High_fallats,'Pass', 'Passes_fallats_High')

In [37]:
merged_df_passes_High = pd.merge(passes_totals_High, passes_completats_High, on='playerId', how='left')
merged_df_passes_High = pd.merge(merged_df_passes_High, passes_fallats_High, on='playerId', how='left')
# Fill NaN values with 0 (for players who didn't have passes completed or failed)
merged_df_passes_High = merged_df_passes_High.fillna(0)
merged_df_passes_High[['Passes_completed_High', 'Passes_fallats_High']] = merged_df_passes_High[['Passes_completed_High', 'Passes_fallats_High']].astype(int)
merged_df_passes_High['%_pass_success_High'] = round((merged_df_passes_High['Passes_completed_High'] / merged_df_passes_High['Total_passes_High']) * 100,2)

In [38]:
display(merged_df_passes_High)

,playerId,Total_passes_High,Passes_completed_High,Passes_fallats_High,%_pass_success_High
0,2lvit204llltk13iglsa2tjah,1,0,1,0.00
1,3sc349yey596xp2j6xlyt0frp,20,16,4,80.00
2,3vx94h32ahujciraspdayj9t6,11,10,1,90.91
3,4u281v53ges3kimtgac0tidm2,17,10,7,58.82
4,5ak9fwtqlr2pll0nsv5br7p7u,10,5,5,50.00
5,5qgc6zjc38a5xjl35gs7h3vu1,19,13,6,68.42
6,6ekdnbnk56xlxforb5owt3dn9,2,1,1,50.00
7,6j0ogojh2b7poyceg7i3k09yi,28,24,4,85.71
8,6u2ob6fv950r1qve8uejkq2uh,25,20,5,80.00
9,72d5uxwcmvhd6mzthxuvev1sl,10,10,0,100.00


#### Taules amb totals/attempted/failed i % conversion rate per cada pressure i en total, i totals amb % unicament

In [56]:
# Merge the DataFrames
#merged_df_passes_TP -> TP = Total and Pressure
merged_df_passes_TP = pd.merge(merged_df_passes, merged_df_passes_NaN, on='playerId', how='left')
merged_df_passes_TP = pd.merge(merged_df_passes_TP, merged_df_passes_Low, on='playerId', how='left')
merged_df_passes_TP = pd.merge(merged_df_passes_TP, merged_df_passes_Medium, on='playerId', how='left')
merged_df_passes_TP = pd.merge(merged_df_passes_TP, merged_df_passes_High, on='playerId', how='left')


In [57]:
display(merged_df_passes_TP)

,playerId,Total_passes,Passes_completed,Passes_failed,%_pass_success,Total_passes_NaN,Passes_completed_NaN,Passes_fallats_NaN,%_pass_success_NaN,Total_passes_Low,Passes_completed_Low,Passes_fallats_Low,%_pass_success_Low,Total_passes_Medium,Passes_completed_Medium,Passes_fallats_Medium,%_pass_success_Medium,Total_passes_High,Passes_completed_High,Passes_fallats_High,%_pass_success_High
0,2lvit204llltk13iglsa2tjah,2,1,1,50.00,1.0,1.0,0.0,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,0.00
1,3sc349yey596xp2j6xlyt0frp,54,44,10,81.48,21.0,17.0,4.0,80.95,NaN,NaN,NaN,NaN,13.0,11.0,2.0,84.62,20,16,4,80.00
2,3vx94h32ahujciraspdayj9t6,19,17,2,89.47,4.0,4.0,0.0,100.00,1.0,1.0,0.0,100.00,3.0,2.0,1.0,66.67,11,10,1,90.91
3,4u281v53ges3kimtgac0tidm2,52,42,10,80.77,20.0,18.0,2.0,90.00,4.0,4.0,0.0,100.00,11.0,10.0,1.0,90.91,17,10,7,58.82
4,5ak9fwtqlr2pll0nsv5br7p7u,20,12,8,60.00,5.0,4.0,1.0,80.00,2.0,1.0,1.0,50.00,3.0,2.0,1.0,66.67,10,5,5,50.00
5,5qgc6zjc38a5xjl35gs7h3vu1,36,26,10,72.22,7.0,5.0,2.0,71.43,3.0,2.0,1.0,66.67,7.0,6.0,1.0,85.71,19,13,6,68.42
6,6ekdnbnk56xlxforb5owt3dn9,47,39,8,82.98,44.0,37.0,7.0,84.09,NaN,NaN,NaN,NaN,1.0,1.0,0.0,100.00,2,1,1,50.00
7,6j0ogojh2b7poyceg7i3k09yi,64,58,6,90.62,15.0,14.0,1.0,93.33,8.0,8.0,0.0,100.00,13.0,12.0,1.0,92.31,28,24,4,85.71
8,6u2ob6fv950r1qve8uejkq2uh,59,50,9,84.75,7.0,7.0,0.0,100.00,9.0,7.0,2.0,77.78,18.0,16.0,2.0,88.89,25,20,5,80.00
9,72d5uxwcmvhd6mzthxuvev1sl,38,36,2,94.74,23.0,21.0,2.0,91.30,1.0,1.0,0.0,100.00,4.0,4.0,0.0,100.00,10,10,0,100.00


In [58]:
selected_columns = ['playerId',
                    'Total_passes', '%_pass_success',
                    'Total_passes_NaN', '%_pass_success_NaN',
                    'Total_passes_Low', '%_pass_success_Low',
                    'Total_passes_Medium', '%_pass_success_Medium',
                    'Total_passes_High', '%_pass_success_High']

merged_df_passes_pressure_overall = merged_df_passes_TP[selected_columns].copy()

In [59]:
display(merged_df_passes_pressure_overall)

,playerId,Total_passes,%_pass_success,Total_passes_NaN,%_pass_success_NaN,Total_passes_Low,%_pass_success_Low,Total_passes_Medium,%_pass_success_Medium,Total_passes_High,%_pass_success_High
0,2lvit204llltk13iglsa2tjah,2,50.00,1.0,100.00,NaN,NaN,NaN,NaN,1,0.00
1,3sc349yey596xp2j6xlyt0frp,54,81.48,21.0,80.95,NaN,NaN,13.0,84.62,20,80.00
2,3vx94h32ahujciraspdayj9t6,19,89.47,4.0,100.00,1.0,100.00,3.0,66.67,11,90.91
3,4u281v53ges3kimtgac0tidm2,52,80.77,20.0,90.00,4.0,100.00,11.0,90.91,17,58.82
4,5ak9fwtqlr2pll0nsv5br7p7u,20,60.00,5.0,80.00,2.0,50.00,3.0,66.67,10,50.00
5,5qgc6zjc38a5xjl35gs7h3vu1,36,72.22,7.0,71.43,3.0,66.67,7.0,85.71,19,68.42
6,6ekdnbnk56xlxforb5owt3dn9,47,82.98,44.0,84.09,NaN,NaN,1.0,100.00,2,50.00
7,6j0ogojh2b7poyceg7i3k09yi,64,90.62,15.0,93.33,8.0,100.00,13.0,92.31,28,85.71
8,6u2ob6fv950r1qve8uejkq2uh,59,84.75,7.0,100.00,9.0,77.78,18.0,88.89,25,80.00
9,72d5uxwcmvhd6mzthxuvev1sl,38,94.74,23.0,91.30,1.0,100.00,4.0,100.00,10,100.00


#### Taula amb Passes totals intentats per Pressure

In [39]:
# Pivot the DataFrame
passes_count_pivot = df_passes_pressure.pivot_table(
    index='playerId',
    columns='pressure_received',
    values='eventId',
    aggfunc='count',
    fill_value=0
)

# Flatten the columns
passes_count_pivot.columns = passes_count_pivot.columns.map('_'.join)

# Reset the index
passes_count_pivot.reset_index(inplace=True)

passes_count_pivot.columns = ['playerId', 'pressure_NaN', 'pressure_High', 'pressure_Low', 'pressure_Medium']
passes_count_pivot.columns.name = None
passes_count_pivot = passes_count_pivot[['playerId', 'pressure_NaN', 'pressure_Low', 'pressure_Medium', 'pressure_High']]

In [40]:
display(passes_count_pivot)

,playerId,pressure_NaN,pressure_Low,pressure_Medium,pressure_High
0,2lvit204llltk13iglsa2tjah,1,0,0,1
1,3sc349yey596xp2j6xlyt0frp,21,0,13,20
2,3vx94h32ahujciraspdayj9t6,4,1,3,11
3,4u281v53ges3kimtgac0tidm2,20,4,11,17
4,5ak9fwtqlr2pll0nsv5br7p7u,5,2,3,10
5,5qgc6zjc38a5xjl35gs7h3vu1,7,3,7,19
6,6ekdnbnk56xlxforb5owt3dn9,44,0,1,2
7,6j0ogojh2b7poyceg7i3k09yi,15,8,13,28
8,6u2ob6fv950r1qve8uejkq2uh,7,9,18,25
9,72d5uxwcmvhd6mzthxuvev1sl,23,1,4,10
